# RAG Explore: Interaktives Experimentieren mit Retrieval-Augmented Generation

In diesem Notebook kannst du alle wichtigen Parameter einer RAG-Pipeline mit LangChain selbst anpassen und die Auswirkungen direkt beobachten.

**Was kannst du hier explorieren?**
- Chunkgröße & Overlap
- Prompt-Formulierung
- Anzahl der Top-k Chunks für das Retrieval

**Ziel:**
Verstehe, wie sich die Parameter auf die Qualität der Antworten und die Auswahl der Chunks auswirken. Probiere verschiedene Kombinationen aus und diskutiere die Ergebnisse!

---


In [ ]:
# !pip install langchain langchain_community langchain_openai openai pypdf python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import numpy as np
import textwrap

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import chromadb

# API-Key aus .env laden
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


## 1. PDFs einlesen

Hier werden die drei Lieferantenberichte geladen und als Dokumente (je Seite) gespeichert.

In [ ]:
pdf_dir = "data/PDFs"
all_docs = []

for fn in os.listdir(pdf_dir):
    if not fn.endswith(".pdf"): continue
    loader = PyPDFLoader(os.path.join(pdf_dir, fn))
    docs = loader.load()
    supplier_name = fn.replace(".pdf", "")
    for d in docs:
        d.metadata["source_file"] = fn
        d.metadata["supplier"] = supplier_name
    all_docs.extend(docs)

print(f"{len(all_docs)} Seiten aus PDF-Dateien geladen.")


6 Seiten insgesamt geladen.


## 2. Chunking – **Experimentier-Bereich!**

Ändere hier Chunkgröße und Overlap und sieh, wie sich die Chunks ändern!

In [ ]:
# <<< Hier Chunkgröße/Overlap anpassen >>>
chunk_size = 800      # z.B. 500, 800, 1200
chunk_overlap = 100   # z.B. 50, 100, 200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)
split_docs = text_splitter.split_documents(all_docs)

chunk_lens = [len(d.page_content.split()) for d in split_docs]
plt.hist(chunk_lens, bins=20, color='royalblue')
plt.title("Wortlängen der Chunks")
plt.xlabel("Wortanzahl pro Chunk")
plt.ylabel("Anzahl Chunks")
plt.show()

print(f"{len(split_docs)} Chunks (Größe: {chunk_size}, Overlap: {chunk_overlap})")


10 Chunks (Größe: 800, Overlap: 100)


## 3. Embeddings und Vektorstore (Chroma)

Wandle die Chunks in Embeddings um und speichere sie im Vektorstore.

In [ ]:
embeddings = OpenAIEmbeddings()
client = chromadb.PersistentClient(path="db/")
if "chunks" in [c.name for c in client.list_collections()]:
    client.delete_collection("chunks")

vectordb = Chroma(client=client, collection_name="chunks",
                   embedding_function=embeddings)
vectordb.add_documents(split_docs)
print("✅ Vektorstore erstellt (Duplikate gelöscht)")


## 4. Retriever & Chatbot – **Experimentier-Bereich!**

Passe hier top_k (wie viele relevante Chunks?) an und formuliere den Prompt beliebig um!

In [ ]:
# <<< Hier top_k und Prompt anpassen >>>
top_k = 4
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""Du bist ein Rechercheassistent für Lieferantenberichte. Antworte nur auf Basis des Kontexts und fasse präzise zusammen.
Kontext:
{context}
Frage: {question}
Antwort:"""
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever = vectordb.as_retriever(search_kwargs={"k": top_k})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)


## 5. Eigene Frage testen – **Experimentier-Bereich!**

Gib hier deine eigene Frage ein!

In [ ]:
frage = "Wie geht Sparfuchs GmbH mit Reklamationen um?"  # Beliebige Frage eintragen
result = qa_chain({"query": frage})
print("Frage:", frage)
print("Antwort:", result["result"])

print(f"\nTop-{top_k} relevante Chunks:")
for idx, doc in enumerate(result["source_documents"][:top_k], 1):
    sup = doc.metadata.get("supplier", "unbekannt")
    src = doc.metadata.get("source_file", "unbekannt")
    print(f"[{idx}] {sup} ({src}):")
    print(textwrap.shorten(doc.page_content.strip(), width=300, placeholder="..."))


Frage: Wie geht Sparfuchs GmbH mit Reklamationen um?
Antwort: Der Bericht enthält keine spezifischen Informationen darüber, wie Sparfuchs GmbH mit Reklamationen umgeht. Es wird jedoch erwähnt, dass die Zusammenarbeit mit Sparfuchs von Herausforderungen in Bezug auf Qualität und Transparenz geprägt ist, was darauf hindeutet, dass es möglicherweise Schwierigkeiten bei der Bearbeitung von Reklamationen geben könnte. Weitere Details zu diesem Thema sind im Kontext nicht vorhanden.


In [ ]:
frage = "Wie geht Sparfuchs GmbH mit Reklamationen um?" # <<< Hier beliebige Frage stellen!
result = qa_chain({"query": frage})
print("Frage:", frage)
print("Antwort:", result["result"])


In [ ]:
frage = "Wie geht Sparfuchs GmbH mit Reklamationen um?" # <<< Hier beliebige Frage stellen!
result = qa_chain({"query": frage})
print("Frage:", frage)
print("Antwort:", result["result"])


In [ ]:
frage = "Wie geht Sparfuchs GmbH mit Reklamationen um?" # <<< Hier beliebige Frage stellen!
result = qa_chain({"query": frage})
print("Frage:", frage)
print("Antwort:", result["result"])


In [ ]:
frage = "Wie geht Sparfuchs GmbH mit Reklamationen um?" # <<< Hier beliebige Frage stellen!
result = qa_chain({"query": frage})
print("Frage:", frage)
print("Antwort:", result["result"])


## 6. Aufgaben/Tipps

- Ändere die Chunkgröße und vergleiche die Resultate.
- Variiere top_k.
- Passe den Prompt kreativ an!
- Teste verschiedene eigene Fragen.
- Diskutiere mit anderen, welche Konfiguration am besten ist!
